# Baseline Experiment: Sample Submission + Fix Direction

This notebook:
1. Scores the sample submission to establish baseline
2. Applies fix_direction optimization to improve bounding boxes
3. Validates for overlaps
4. Creates final submission

In [1]:
import numpy as np
import pandas as pd
from decimal import Decimal, getcontext
from shapely import affinity
from shapely.geometry import Polygon
from shapely.ops import unary_union
from shapely.strtree import STRtree
from scipy.spatial import ConvexHull
from scipy.optimize import minimize_scalar
import warnings
warnings.filterwarnings('ignore')

getcontext().prec = 30
scale_factor = 1

print('Libraries loaded successfully')

Libraries loaded successfully


In [2]:
# Tree geometry constants
TX = [0, 0.125, 0.0625, 0.2, 0.1, 0.35, 0.075, 0.075, -0.075, -0.075, -0.35, -0.1, -0.2, -0.0625, -0.125]
TY = [0.8, 0.5, 0.5, 0.25, 0.25, 0, 0, -0.2, -0.2, 0, 0, 0.25, 0.25, 0.5, 0.5]

class ChristmasTree:
    """Represents a single, rotatable Christmas tree of a fixed size."""

    def __init__(self, center_x='0', center_y='0', angle='0'):
        """Initializes the Christmas tree with a specific position and rotation."""
        self.center_x = Decimal(str(center_x))
        self.center_y = Decimal(str(center_y))
        self.angle = Decimal(str(angle))

        trunk_w = Decimal('0.15')
        trunk_h = Decimal('0.2')
        base_w = Decimal('0.7')
        mid_w = Decimal('0.4')
        top_w = Decimal('0.25')
        tip_y = Decimal('0.8')
        tier_1_y = Decimal('0.5')
        tier_2_y = Decimal('0.25')
        base_y = Decimal('0.0')
        trunk_bottom_y = -trunk_h

        initial_polygon = Polygon(
            [
                (Decimal('0.0') * scale_factor, tip_y * scale_factor),
                (top_w / Decimal('2') * scale_factor, tier_1_y * scale_factor),
                (top_w / Decimal('4') * scale_factor, tier_1_y * scale_factor),
                (mid_w / Decimal('2') * scale_factor, tier_2_y * scale_factor),
                (mid_w / Decimal('4') * scale_factor, tier_2_y * scale_factor),
                (base_w / Decimal('2') * scale_factor, base_y * scale_factor),
                (trunk_w / Decimal('2') * scale_factor, base_y * scale_factor),
                (trunk_w / Decimal('2') * scale_factor, trunk_bottom_y * scale_factor),
                (-(trunk_w / Decimal('2')) * scale_factor, trunk_bottom_y * scale_factor),
                (-(trunk_w / Decimal('2')) * scale_factor, base_y * scale_factor),
                (-(base_w / Decimal('2')) * scale_factor, base_y * scale_factor),
                (-(mid_w / Decimal('4')) * scale_factor, tier_2_y * scale_factor),
                (-(mid_w / Decimal('2')) * scale_factor, tier_2_y * scale_factor),
                (-(top_w / Decimal('4')) * scale_factor, tier_1_y * scale_factor),
                (-(top_w / Decimal('2')) * scale_factor, tier_1_y * scale_factor),
            ]
        )
        rotated = affinity.rotate(initial_polygon, float(self.angle), origin=(0, 0))
        self.polygon = affinity.translate(rotated,
                                          xoff=float(self.center_x * scale_factor),
                                          yoff=float(self.center_y * scale_factor))

    def clone(self):
        return ChristmasTree(
            center_x=str(self.center_x),
            center_y=str(self.center_y),
            angle=str(self.angle),
        )

print('ChristmasTree class defined')

ChristmasTree class defined


In [3]:
def load_configuration_from_df(n, df):
    """Loads all trees for a given N from the submission DataFrame."""
    group_data = df[df["id"].str.startswith(f"{n:03d}_")]
    trees = []
    for _, row in group_data.iterrows():
        x = str(row["x"])[1:]  # Remove 's' prefix
        y = str(row["y"])[1:]
        deg = str(row["deg"])[1:]
        if x and y and deg:
            trees.append(ChristmasTree(x, y, deg))
    return trees


def get_score(trees, n):
    """Calculates the score (S^2 / N) for a given configuration of trees."""
    if not trees:
        return 0.0
    xys = np.concatenate([np.asarray(t.polygon.exterior.xy).T / float(scale_factor) for t in trees])
    min_x, min_y = xys.min(axis=0)
    max_x, max_y = xys.max(axis=0)
    side_length = max(max_x - min_x, max_y - min_y)
    return side_length**2 / n


def has_overlap(trees):
    """Check if any two ChristmasTree polygons overlap."""
    if len(trees) <= 1:
        return False
    polygons = [t.polygon for t in trees]
    tree_index = STRtree(polygons)
    for i, poly in enumerate(polygons):
        indices = tree_index.query(poly)
        for idx in indices:
            if idx == i:
                continue
            if poly.intersects(polygons[idx]) and not poly.touches(polygons[idx]):
                return True
    return False


def score_and_validate_submission(file_path, max_n=200):
    """Reads a submission CSV, calculates the total score, and checks for overlaps."""
    df = pd.read_csv(file_path)
    total_score = 0.0
    failed_overlap_n = []
    
    print(f"--- Scoring and Validation: {file_path} (N=1 to {max_n}) ---")

    for n in range(1, max_n + 1):
        trees = load_configuration_from_df(n, df)
        if trees:
            current_score = get_score(trees, n)
            total_score += current_score
            if has_overlap(trees):
                failed_overlap_n.append(n)
                print(f"  ❌ N={n:03d}: OVERLAP DETECTED! (Score: {current_score:.6f})")
        
    print(f"\n--- Summary ---")
    if failed_overlap_n:
        print(f"❌ Validation FAILED: Overlaps found in N: {failed_overlap_n}")
        status = "FAILED (Overlaps)"
    else:
        print("✅ Validation SUCCESSFUL: No overlaps detected.")
        status = "SUCCESS"
        
    print(f"**Total Score (Σ S²/N): {total_score:.6f}**")
    
    return {
        "status": status,
        "total_score": total_score,
        "failed_overlap_n": failed_overlap_n
    }

print('Scoring functions defined')

Scoring functions defined


In [4]:
# Score the sample submission to establish baseline
print('Scoring sample submission...')
baseline_result = score_and_validate_submission('/home/data/sample_submission.csv', max_n=200)
print(f"\nBaseline Score: {baseline_result['total_score']:.6f}")

Scoring sample submission...
--- Scoring and Validation: /home/data/sample_submission.csv (N=1 to 200) ---



--- Summary ---
✅ Validation SUCCESSFUL: No overlaps detected.
**Total Score (Σ S²/N): 173.652299**

Baseline Score: 173.652299


In [ ]:
# Fix direction optimization functions
def calculate_bbox_side_at_angle(angle_deg, points):
    """Calculate bounding box side length at a given rotation angle."""
    angle_rad = np.radians(angle_deg)
    c, s = np.cos(angle_rad), np.sin(angle_rad)
    rot_matrix_T = np.array([[c, s], [-s, c]])
    rotated_points = points.dot(rot_matrix_T)
    min_xy = np.min(rotated_points, axis=0)
    max_xy = np.max(rotated_points, axis=0)
    return max(max_xy[0] - min_xy[0], max_xy[1] - min_xy[1])


def optimize_rotation(trees):
    """Find optimal rotation angle to minimize bounding box."""
    all_points = []
    for tree in trees:
        all_points.extend(list(tree.polygon.exterior.coords))
    points_np = np.array(all_points)
    
    # Use convex hull for efficiency
    hull_points = points_np[ConvexHull(points_np).vertices]
    
    initial_side = calculate_bbox_side_at_angle(0, hull_points)
    
    res = minimize_scalar(
        lambda a: calculate_bbox_side_at_angle(a, hull_points),
        bounds=(0.001, 89.999), 
        method='bounded'
    )
    found_angle_deg = res.x
    found_side = res.fun
    
    improvement = initial_side - found_side
    EPSILON = 1e-8
    
    if improvement > EPSILON:
        return Decimal(found_side) / scale_factor, found_angle_deg
    else:
        return Decimal(initial_side) / scale_factor, 0.0


def apply_rotation(trees, angle_deg):
    """Apply rotation to all trees around their collective center."""
    if not trees or abs(angle_deg) < 1e-9:
        return [t.clone() for t in trees]
    
    bounds = [t.polygon.bounds for t in trees]
    min_x = min(b[0] for b in bounds)
    min_y = min(b[1] for b in bounds)
    max_x = max(b[2] for b in bounds)
    max_y = max(b[3] for b in bounds)
    rotation_center = np.array([(min_x + max_x) / 2.0, (min_y + max_y) / 2.0])
    
    angle_rad = np.radians(angle_deg)
    c, s = np.cos(angle_rad), np.sin(angle_rad)
    rot_matrix = np.array([[c, -s], [s, c]])
    
    points = np.array([[float(t.center_x), float(t.center_y)] for t in trees])
    shifted = points - rotation_center
    rotated = shifted.dot(rot_matrix.T) + rotation_center
    
    rotated_trees = []
    for i in range(len(trees)):
        new_tree = ChristmasTree(
            str(rotated[i, 0]), 
            str(rotated[i, 1]),
            str(float(trees[i].angle) + angle_deg)
        )
        rotated_trees.append(new_tree)
    return rotated_trees

print('Fix direction functions defined')

In [ ]:
# Apply fix_direction to all configurations
def fix_direction_all(input_path, output_path, passes=1):
    """Apply fix_direction optimization to all configurations."""
    df = pd.read_csv(input_path)
    
    # Parse all configurations
    dict_of_tree_list = {}
    dict_of_side_length = {}
    
    for n in range(1, 201):
        trees = load_configuration_from_df(n, df)
        if trees:
            dict_of_tree_list[f"{n:03d}"] = trees
            # Calculate initial side length
            xys = np.concatenate([np.asarray(t.polygon.exterior.xy).T for t in trees])
            min_xy = xys.min(axis=0)
            max_xy = xys.max(axis=0)
            dict_of_side_length[f"{n:03d}"] = Decimal(max(max_xy[0] - min_xy[0], max_xy[1] - min_xy[1]))
    
    # Calculate initial score
    initial_score = sum(float(v**2) / int(k) for k, v in dict_of_side_length.items())
    print(f"Initial score: {initial_score:.6f}")
    
    # Apply fix_direction for each pass
    for pass_num in range(passes):
        improved_count = 0
        for group_id in dict_of_tree_list.keys():
            trees = dict_of_tree_list[group_id]
            if len(trees) < 2:
                continue
            
            current_side = dict_of_side_length[group_id]
            best_side, best_angle = optimize_rotation(trees)
            
            if best_side < current_side:
                rotated_trees = apply_rotation(trees, best_angle)
                dict_of_tree_list[group_id] = rotated_trees
                dict_of_side_length[group_id] = best_side
                improved_count += 1
        
        new_score = sum(float(v**2) / int(k) for k, v in dict_of_side_length.items())
        print(f"Pass {pass_num + 1}: Improved {improved_count} configs, Score: {new_score:.6f}")
    
    # Save to output
    rows = []
    for group_id in sorted(dict_of_tree_list.keys()):
        trees = dict_of_tree_list[group_id]
        for i, tree in enumerate(trees):
            rows.append({
                'id': f"{group_id}_{i}",
                'x': f"s{float(tree.center_x):.6f}",
                'y': f"s{float(tree.center_y):.6f}",
                'deg': f"s{float(tree.angle):.6f}"
            })
    
    result_df = pd.DataFrame(rows)
    result_df.to_csv(output_path, index=False)
    
    final_score = sum(float(v**2) / int(k) for k, v in dict_of_side_length.items())
    print(f"\nFinal score after fix_direction: {final_score:.6f}")
    return final_score

print('fix_direction_all function defined')

In [ ]:
# Apply fix_direction optimization
print('Applying fix_direction optimization...')
final_score = fix_direction_all(
    '/home/data/sample_submission.csv',
    '/home/code/experiments/001_baseline/submission_fixed.csv',
    passes=3
)

In [ ]:
# Validate the optimized submission
print('\nValidating optimized submission...')
optimized_result = score_and_validate_submission(
    '/home/code/experiments/001_baseline/submission_fixed.csv', 
    max_n=200
)
print(f"\nOptimized Score: {optimized_result['total_score']:.6f}")

In [ ]:
# Copy to submission folder
import shutil
shutil.copy(
    '/home/code/experiments/001_baseline/submission_fixed.csv',
    '/home/submission/submission.csv'
)
print('Submission saved to /home/submission/submission.csv')

# Summary
print(f"\n=== SUMMARY ===")
print(f"Baseline Score: {baseline_result['total_score']:.6f}")
print(f"Optimized Score: {optimized_result['total_score']:.6f}")
print(f"Improvement: {baseline_result['total_score'] - optimized_result['total_score']:.6f}")
print(f"Target: 68.922808")